In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import math

In [2]:
price_df = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\RA_Cryptocurrency\Data\Input Data\price_data.csv", encoding="ISO-8859-1")
price_df

,Unnamed: 0,ID,timestamps,date_time,price,market_caps,volumes
0,0,ethereum,1637884800000,2021-11-26,4515.843300,5.350824e+11,2.024153e+10
1,1,ethereum,1637971200000,2021-11-27,4048.312844,4.794143e+11,2.927673e+10
2,2,ethereum,1638057600000,2021-11-28,4084.088486,4.839973e+11,1.650051e+10
3,3,ethereum,1638144000000,2021-11-29,4290.091863,5.083852e+11,1.740861e+10
4,4,ethereum,1638230400000,2021-11-30,4444.528280,5.273311e+11,2.015092e+10
...,...,...,...,...,...,...,...
6411,396,cosmos,1672099200000,2022-12-27,9.165806,2.685435e+09,7.984043e+07
6412,397,cosmos,1672185600000,2022-12-28,9.217066,2.692638e+09,9.948979e+07
6413,398,cosmos,1672272000000,2022-12-29,9.121968,2.670882e+09,1.163803e+08
6414,399,cosmos,1672358400000,2022-12-30,9.342944,2.736200e+09,1.141521e+08


In [3]:
price_df

,Unnamed: 0,ID,timestamps,date_time,price,market_caps,volumes
0,0,ethereum,1637884800000,2021-11-26,4515.843300,5.350824e+11,2.024153e+10
1,1,ethereum,1637971200000,2021-11-27,4048.312844,4.794143e+11,2.927673e+10
2,2,ethereum,1638057600000,2021-11-28,4084.088486,4.839973e+11,1.650051e+10
3,3,ethereum,1638144000000,2021-11-29,4290.091863,5.083852e+11,1.740861e+10
4,4,ethereum,1638230400000,2021-11-30,4444.528280,5.273311e+11,2.015092e+10
...,...,...,...,...,...,...,...
6411,396,cosmos,1672099200000,2022-12-27,9.165806,2.685435e+09,7.984043e+07
6412,397,cosmos,1672185600000,2022-12-28,9.217066,2.692638e+09,9.948979e+07
6413,398,cosmos,1672272000000,2022-12-29,9.121968,2.670882e+09,1.163803e+08
6414,399,cosmos,1672358400000,2022-12-30,9.342944,2.736200e+09,1.141521e+08


In [4]:
price_df = price_df.rename(columns={"Unnamed: 0": "day"})

In [5]:
for x in price_df.index:
    price_df["date_time"][x] = datetime.strptime(price_df["date_time"][x], '%Y-%m-%d')

<ipython-input-5-b8d0c318e07f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df["date_time"][x] = datetime.strptime(price_df["date_time"][x], '%Y-%m-%d')


In [6]:
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6416 entries, 0 to 6415
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   day          6416 non-null   int64  
 1   ID           6416 non-null   object 
 2   timestamps   6416 non-null   int64  
 3   date_time    6416 non-null   object 
 4   price        6416 non-null   float64
 5   market_caps  6416 non-null   float64
 6   volumes      6416 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 351.0+ KB


In [7]:
def variance(slice_df, ddof=0): #ddof = 0 for population, =1 for sample variance 
    data = slice_df["%change"]
    n = len(data)
    if(n != 0):
        mean = sum(data) / n
        return sum((x - mean) ** 2 for x in data) / (n - ddof)
    else:
        return 144

In [8]:
def st_dev(slice_df):
    var = variance(slice_df)
    std_dev = math.sqrt(var)
    return std_dev

In [9]:
def st_dev_calculator(price_df):
    temp_df = price_df
    slice_df = pd.DataFrame()
    result_df = pd.DataFrame()
    temp_df["price_dif"] = 0.0
    temp_df["%change"] = 0.0
    temp_df["st_dev"] = 0.0
    temp_df[">st_dev"] = 0
    
    for x in temp_df.index:
        if(x != 0):
            x_price = temp_df["price"][x]
            pre_price = temp_df["price"][x-1]
            temp_df["price_dif"][x] = x_price-pre_price
            x_price_dif = temp_df["price_dif"][x]
            temp_df["%change"][x] = x_price_dif/pre_price
            x_rate = temp_df["%change"][x]
            
            if(temp_df["day"][x] >= 201):
                interval = list(range(x-200, x)) #interval is equal to [x-200, x)
                slice_df = temp_df[temp_df.index.isin(interval)]
                st_dev_x = st_dev(slice_df)
                temp_df["st_dev"][x] = st_dev_x
                
                if(abs(x_rate) > st_dev_x):
                    temp_df[">st_dev"][x] = 1
                
                result_df = result_df.append(temp_df.iloc[[x]])
                
    return result_df

In [10]:
price_df_v2 = st_dev_calculator(price_df)

<ipython-input-9-0aa51e9324ca>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["price_dif"][x] = x_price-pre_price
<ipython-input-9-0aa51e9324ca>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["%change"][x] = x_price_dif/pre_price
<ipython-input-9-0aa51e9324ca>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["st_dev"][x] = st_dev_x
<ipython-input-9-0aa51e9324ca>:26: SettingWithCopyWarning: 
A value is try

In [11]:
price_df_v2 = price_df_v2.reset_index()
price_df_v2 = price_df_v2.drop(["index"], axis=1)

In [12]:
price_df_v2

,day,ID,timestamps,date_time,price,market_caps,volumes,price_dif,%change,st_dev,>st_dev
0,201,ethereum,1655251200000,2022-06-15 00:00:00,1214.866265,1.468668e+11,3.277225e+10,9.270979,0.007690,0.042284,0
1,202,ethereum,1655337600000,2022-06-16 00:00:00,1230.364335,1.487913e+11,3.967981e+10,15.498070,0.012757,0.041720,0
2,203,ethereum,1655424000000,2022-06-17 00:00:00,1068.603575,1.292002e+11,1.801651e+10,-161.760760,-0.131474,0.041728,1
3,204,ethereum,1655510400000,2022-06-18 00:00:00,1087.813924,1.320571e+11,1.406726e+10,19.210350,0.017977,0.042483,0
4,205,ethereum,1655596800000,2022-06-19 00:00:00,995.252569,1.201489e+11,2.111267e+10,-92.561355,-0.085089,0.042413,1
...,...,...,...,...,...,...,...,...,...,...,...
3195,396,cosmos,1672099200000,2022-12-27 00:00:00,9.165806,2.685435e+09,7.984043e+07,0.140715,0.015592,0.056996,0
3196,397,cosmos,1672185600000,2022-12-28 00:00:00,9.217066,2.692638e+09,9.948979e+07,0.051261,0.005593,0.057004,0
3197,398,cosmos,1672272000000,2022-12-29 00:00:00,9.121968,2.670882e+09,1.163803e+08,-0.095098,-0.010318,0.056722,0
3198,399,cosmos,1672358400000,2022-12-30 00:00:00,9.342944,2.736200e+09,1.141521e+08,0.220976,0.024225,0.056503,0


In [13]:
price_df_v2.to_excel(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\RA_Cryptocurrency\Data\price_data_v2.xlsx")
price_df_v2.to_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\RA_Cryptocurrency\Data\Input Data\price_data_v2.csv")